In [1]:
import random
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
# set working directory - note: all code runs from the src folder
wrk_dir = os.getcwd()
# set data path
data_path = wrk_dir + '\\' + 'data' + '\\'

In [ ]:
# overall params
np.random.seed(2022)
n = 5000

# (hyper)parameters
lambda_1 = 10000
lambda_2 = 2500
beta_1 = (3/10)
beta_2 = 5
#epsilon = 225000

# introduce the confounder z
z = np.random.normal(loc=0.0, scale=1.75, size=n)
lambda_z1 = 1000
lambda_z2 = 3*lambda_z1

# input to the errors
u1 = lambda_1*np.random.poisson(lam=10, size=n)            + lambda_z1*z
u2 = lambda_2*np.random.normal(loc=0.0, scale=1.0, size=n) + lambda_z2*z

# annual salary
x1 = u1
# account balance
x2 = beta_1*x1 + u2

# loan approval
y = np.sign(x1 + beta_2*x2 - 225000)

In [ ]:
# store data for testing RStan 
d2 = {'LoanApproval': y, 
      'AnnualSalary': x1, 
      'AccountBalance': x2,
      'u1': u1,
      'u2': u2,
      'z': z}
data2 = pd.DataFrame(d2)
data2.head(5)

In [ ]:
# check for negative values
print(data2.shape)
data2 = data2[(data2['AnnualSalary'] >= 0) & (data2['AccountBalance'] >= 0)]
print(data2.shape)

In [ ]:
#from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True, normalize=False)

In [ ]:
x = np.array(data2['AnnualSalary'].copy()).reshape((-1, 1))
print(x.shape)
x

y = np.array(data2['AccountBalance'].copy())
print(y.shape)
y

In [ ]:
# fit model
model.fit(x, y)

hat_beta1 = model.coef_[0]
print(hat_beta1)

hat_beta0 = model.intercept_
print(hat_beta0)

In [ ]:
# check the abduction step!!!
plt.hist(round(data2['AccountBalance'] - model.predict(x), 2)) #hat_u2
plt.hist(data2['u2']) #u2

In [ ]:
#from sklearn.linear_model import LinearRegression
model2 = LinearRegression(fit_intercept=True, normalize=False)

In [ ]:
x2 = np.array(data2[['AnnualSalary', 'z']].copy())#.reshape((1, -1))
print(x2.shape)
x2

# fit model with 'observed' confounder
model2.fit(x2, y)

hat_beta1 = model2.coef_[0]
print(hat_beta1)

hat_beta2 = model2.coef_[1]
print(hat_beta2)

hat_beta0 = model2.intercept_
print(hat_beta0)


In [ ]:
# the coefficients
model2.coef_

# check the counterfactuals
scf_data = data2[['AnnualSalary', 'AccountBalance', 'u1', 'u2', 'z']].copy()
scf_data.head(5)

scf_data['hat_AccountBalance'] = model2.predict(x2)
scf_data.head(5)

scf_data['hat_u1'] = scf_data['AnnualSalary']
scf_data['hat_u2'] = round(scf_data['AccountBalance'] - model2.predict(x2), 2)
scf_data.head(5)

# under this, we would overestimate u2
plt.hist(scf_data['u2'])
plt.hist(scf_data['hat_u2'])

In [ ]:
# store in data folder
data.to_csv(data_path + '\\' + 'namehere.csv', sep='|', index=False)